## check train files

In [20]:
pip install -U ultralytics

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [21]:
%cd /mnt/disks/data1/aihub/Training
!git clone https://github.com/ultralytics/ultralytics

/mnt/disks/data1/aihub/Training
Cloning into 'ultralytics'...
remote: Enumerating objects: 13102, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (377/377), done.
remote: Total 13102 (delta 150), reused 375 (delta 90), pack-reused 12631
Receiving objects: 100% (13102/13102), 7.85 MiB | 5.87 MiB/s, done.
Resolving deltas: 100% (8660/8660), done.


In [2]:
%cd /mnt/disks/data1/aihub/Training/ultralytics
!pip install -r requirements.txt

/mnt/disks/data1/aihub/Training/ultralytics
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install PyYAML

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from pathlib import Path
from typing import Union, Optional, List, Tuple
from fastai.vision.all import *
import xml.etree.ElementTree as ET
from collections import defaultdict
from dataclasses import dataclass
import zipfile

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [29]:
target_name = "Modified_Data"
dataset_path = Path(f"/mnt/disks/data1/aihub/Training/{target_name}")

def count_categories(dataset_path):
    category_counts = defaultdict(int)

    for file in dataset_path.glob("**/*"):
        if file.suffix == ".xml":
            tree = ET.parse(file)
            root = tree.getroot()
            for obj in root.findall('object'):
                category = obj.find('property/category').text
                category_counts[category] += 1

    for category, count in category_counts.items():
        print(f"{category}: {count}")

count_categories(dataset_path)

기타부유물: 21321
선박: 46712
부표: 4653
등대: 7172
어망부표: 2013
해상풍력: 6275
﻿선박: 85


In [30]:
from IPython.display import display
from ipywidgets import widgets, interact

dataset_path = Path(f"/mnt/disks/data1/aihub/Training/{target_name}")
box_sizes = []
files = []

for file in dataset_path.glob("**/*"):
    if file.suffix == ".xml":
        tree = ET.parse(file)
        root = tree.getroot()
        for obj in root.findall('object'):
            bbox = obj.find('bbox').text # "[x, y, w, h]"
            bbox = bbox.replace("[", "").replace("]", "")
            bbox = bbox.split(",")
            bbox = [float(x) for x in bbox]
            size = bbox[2] * bbox[3]
            if size > 4000: # 객체 크기가 4000 이상인 경우만 선별
                file = file.with_suffix(".jpg") # change extension
                files.append(file)
            
            box_sizes.append(size)

# delete duplicates
files = list(set(files))
files = files[:50] # 확인 할 이미지 개수

# average box size
print(f"average size: {sum(box_sizes) / len(box_sizes)}")

# display images using slider
@interact(idx=widgets.IntSlider(min=0, max=len(files)-1, step=1, value=0))
def display_image(idx):
    img = PILImage.open(imgs[idx])
    display(img)

average size: 131757.72061973682


interactive(children=(IntSlider(value=0, description='idx', max=49), Output()), _dom_classes=('widget-interact…

In [31]:
import xml.etree.ElementTree as ET

dataset_path = Path(f"/mnt/disks/data1/aihub/Training/{target_name}")
label_xml = []

categories = {
    "선박": "0",
    "부표": "1",
    "기타부유물": "2",
    "등대": "3",
    "어망부표": "4",
}

def normalize_bbox(bbox, width, height):
    x, y, w, h = bbox
    x = ((x+(w/2)) / 3840) 
    y = ((y+(h/2)) / 2160) 
    w = (w / 3840) 
    h = (h / 2160) 
    return x, y, w, h

# scan the jpg files recursively. path is the absolute path from the dataset_path
# label file format is <category> <x> <y> <w> <h>
for file in dataset_path.glob("**/*"):
    if file.suffix == '.xml':
        labels = []
        tree = ET.parse(file)
        root = tree.getroot()
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)
        for obj in root.findall('object'):
            label = obj.find('bbox').text # "[x, y, w, h]"
            label = label.replace("[", "").replace("]", "")
            label = label.split(",")
            label = [float(x) for x in label]
            labels.append(label)
        
        with open(file.with_suffix('.txt'), 'w') as f:
            for obj, label in zip(root.findall('object'), labels):
                category = obj.find('property/category').text
                if category not in categories:
                    continue  # Skip this category and go to the next label

                category = categories[category]
                x, y, w, h = normalize_bbox(label, width, height)
                f.write(f"{category} {x} {y} {w} {h}\n")


In [32]:
import random

# record the name of images on the path
# saves the list of the 'jpg' files on the 'images.txt' file
dataset_path = Path(f"/mnt/disks/data1/aihub/Training/{target_name}")
image_files = []

# scan the jpg files recursively. path is the absolute path from the dataset_path
for file in dataset_path.glob("**/*"):
    if file.suffix == ".jpg":
        image_files.append(file)

# save the list of the image files
with open("images.txt", "w") as f:
    for file in image_files:
        f.write(str(file) + "\n")

# separate the images.txt into train.txt, valid.txt, and test.txt
with open("images.txt", "r") as f:
    lines = f.readlines()

random.seed(42)
random.shuffle(lines)

with open("train.txt", "w") as f:
    for line in lines[:int(len(lines) * 0.7)]:
        f.write(line)

with open("valid.txt", "w") as f:
    for line in lines[int(len(lines) * 0.7):int(len(lines) * 0.85)]:
        f.write(line)

with open("test.txt", "w") as f:
    for line in lines[int(len(lines) * 0.85):]:
        f.write(line)

# copy the train.txt, valid.txt, and test.txt to the dataset_path
!cp images.txt {dataset_path}
!cp train.txt {dataset_path}
!cp valid.txt {dataset_path}
!cp test.txt {dataset_path}


In [33]:
import yaml

with open(f'/mnt/disks/data1/aihub/Training/{target_name}/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)


with open(f'/mnt/disks/data1/aihub/Training/{target_name}/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'path': '/mnt/disks/data1/aihub/Training/Modified_Data', 'train': 'train.txt', 'val': 'valid.txt', 'test': 'test.txt', 'names': {0: '선박', 1: '부표', 2: '기타부유물', 3: '등대', 4: '어망부표'}}
{'path': '/mnt/disks/data1/aihub/Training/Modified_Data', 'train': 'train.txt', 'val': 'valid.txt', 'test': 'test.txt', 'names': {0: '선박', 1: '부표', 2: '기타부유물', 3: '등대', 4: '어망부표'}}


In [54]:
from ultralytics import YOLO

# Load a model
model = YOLO('model/yolov8l.pt')
model.train(data=f'/mnt/disks/data1/aihub/Training/{target_name}/data.yaml', epochs=20, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.141 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.137 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100-SXM4-40GB, 40377MiB)
engine/trainer: task=detect, mode=train, model=model/yolov8l.pt, data=/mnt/disks/data1/aihub/Training/동해_묵호항_1구역_BOX/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, aug

KeyboardInterrupt: 